In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as mpl
mpl.use('Agg')
%matplotlib inline
import simplejson
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, Normalizer
import brewer2mpl
import time
import pickle
import re
import requests
import zipfile
import StringIO
import xml.etree.ElementTree as ET
import urllib2
import os

In [2]:
## Get data from the server
url_xml = urllib2.urlopen('https://www.capitalbikeshare.com/data/stations/bikeStations.xml').read()
root = ET.fromstring(url_xml)

terminalName = []
name = []
nbBikes = []
nbEmptyDocks = []

for station in root.findall('station'):
    name.append(station.find('name').text)
    terminalName.append(station.find('terminalName').text)

## Make the bikes data into a dataframe
stations = pd.DataFrame(terminalName)
stations.columns = ['terminalName']
stations['name'] = pd.Series(name, index=stations.index)
new_bikes = stations[['terminalName', 'name']]
new_bikes['name'] = new_bikes.name.apply(lambda x: str(x).lower().strip())
new_bikes.set_index('name', drop = True, inplace = True)

In [7]:
## Get historical rides data for 2011-2016
urls = ['https://s3.amazonaws.com/capitalbikeshare-data/2011-Q1-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2011-Q2-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2011-Q3-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2011-Q4-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2012-Q1-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2012-Q2-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2012-Q3-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2012-Q4-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2013-Q1-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2013-Q2-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2013-Q3-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2013-Q4-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2014-Q1-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2014-Q2-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2014-Q3-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2014-Q4-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2015-Q1-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2015-Q2-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2015-Q3-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2015-Q4-cabi-trip-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2016-Q1-cabi-trip-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2016-Q2-cabi-trips-history-data.zip', 
        'https://s3.amazonaws.com/capitalbikeshare-data/2016-Q3-cabi-trips-history-data.zip', 'https://s3.amazonaws.com/capitalbikeshare-data/2016-Q4-cabi-trips-history-data.zip']
dfs = []

for i, url in enumerate(urls):
    print i
    r = requests.get(url)
    with zipfile.ZipFile(StringIO.StringIO(r.content)) as z:
        df = pd.read_csv(z.open(z.namelist()[0]))
        df.columns = [x.lower() for x in df.columns]
        dfs.append(df)
with open('../data/historical_data/historical_rides_raw_dfs.pkl', 'wb') as f:
    pickle.dump(dfs, f)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [3]:
with open('../data/historical_data/historical_rides_raw_dfs.pkl', 'rb') as f:
    dfs = pickle.load(f)

In [4]:
## Look at columns to clean up column names
# dfs = [x[:] for x in dfs_backup]
cols = []
for i, each in enumerate(dfs):
    print each.columns.tolist()
    print i, each.head(1)
    cols += each.columns.tolist()
    print '################################'
cols = list(set(cols))
print cols

['duration', 'start date', 'end date', 'start station', 'end station', 'bike#', 'member type']
0           duration       start date       end date  \
0  0h 1min. 50sec.  3/31/2011 23:58  4/1/2011 0:00   

                  start station                   end station   bike#  \
0  14th & Harvard St NW (31105)  16th & Harvard St NW (31103)  W00749   

  member type  
0  Registered  
################################
['duration', 'start date', 'end date', 'start station', 'end station', 'bike#', 'member type']
1           duration       start date       end date  \
0  0h 10min. 4sec.  6/30/2011 23:59  7/1/2011 0:09   

                 start station                         end station   bike#  \
0  11th & Kenyon St NW (31102)  Calvert St & Woodley Pl NW (31106)  W00494   

  member type  
0  Registered  
################################
['duration', 'start date', 'end date', 'start station', 'end station', 'bike#', 'member type']
2            duration       start date        end date  \
0

In [5]:
#### Field analysis
# ~~~ Station IDs ~~~
# List of field names: "start station", "end station", "start station number", "end station number"
# 1-4) Start station is the name with the id in parentheses
# 5-18) Start station is just the name, no id
# 19-24) Start station number is the id field (drop "start station" and "end station" from these dfs)

# ~~~ Duration ~~~
# List of field names: "duration", "total duration (ms)", "duration (ms)"
# 1-16) Duration in the format "Xh Xm Xs"
# 17-24) Duration in ms

# ~~~ Datetimes ~~~
# List of field names: "start date", "start time, "end date"
# 1-15) Format: m/d/YYYY H:MM
# 16) Format: YYYY-mm-dd HH:MM
# 17-24) Format: m/d/YYYY H:MM

# ~~~ Bike number ~~~
# List of field names: "bike#", "bike number", "bike #"
# 1-24) Format is a mix of numbers and letters; needs to be label encoded

# ~~~ Account type ~~~
# List of field names: "member type", "type", "bike key", "subscriber type", "subscription type", "account type"
# Registered = "Registered", "Member", or "Subscriber"
# Casual = "Casual"

In [6]:
## Drop columns with only station names from dfs 19-24
for i in range(18, 24):
    dfs[i].drop(['start station', 'end station'], axis = 1, inplace = True)

In [7]:
## Make translation dict of column names
# cols_dict = {
#     'subscription type': 'account_type', 'subscriber type': 'account_type', 'member type': 'account_type', 'account type': 'account_type', 'type': 'account_type', 
#     'end date': 'end_date', 'bike#': 'bike_id', 'start time': 'start_date', 'start date': 'start_date', 'bike key': 'account_type', 'start station number': 'start_id', 
#     'end station number': 'end_id', 'bike number': 'bike_id', 'total duration (ms)': 'duration_ms', 'end station': 'end_id', 'start station': 'start_id', 
#     'duration (ms)': 'duration_ms', 'duration': 'duration_ms', 'member type': 'account_type', 'bike #': 'bike_id'
# }
cols_dict = {
    "start station": 'start_station', "start station number": 'start_station', "end station": 'end_station', "end station number": 'end_station',
    "duration": 'duration', "total duration (ms)": 'duration', "duration (ms)": 'duration', "start date": 'start_datetime', "start time": "start_datetime", "end date": 'end_datetime',
    "bike#": 'bike_id', "bike number": 'bike_id', "bike #": 'bike_id',
    "member type": 'account_type', "type": 'account_type', "bike key": 'account_type', "subscriber type": 'account_type', "subscription type": 'account_type', "account type": 'account_type'
}
## Convert column names to the proper ones
new_cols = []
for i in range(len(dfs)):
    temp_cols = [cols_dict[x] for x in dfs[i].columns]
    dfs[i].columns = temp_cols
    new_cols += temp_cols

In [8]:
## Convert the station fields to station ids for the first 4 dfs
def getStationID(x):
    try:
        return re.findall(r"\D(\d{5})\D", " "+str(x)+" ")[0]
    except:
        return np.nan
for i in range(4):
    dfs[i]['start_station'] = dfs[i]['start_station'].apply(getStationID)
    dfs[i]['end_station'] = dfs[i]['end_station'].apply(getStationID)

In [9]:
## Make station names to ids translator
stations_converter = new_bikes.to_dict()['terminalName']
## Fix multiple station names that correspond to the same station (this will fail when names are updated/changed so view it as an example)
stations_converter['new hampshire ave & t st nw [formerly 16th & u st nw]'] = stations_converter['new hampshire ave & t st nw']
stations_converter['16th & u st nw'] = stations_converter['new hampshire ave & t st nw']
stations_converter['12th & hayes st'] = stations_converter['pentagon city metro / 12th & s hayes st']
stations_converter['12th & hayes st /  pentagon city metro'] = stations_converter['pentagon city metro / 12th & s hayes st']
stations_converter['pentagon city metro / 12th & hayes st'] = stations_converter['pentagon city metro / 12th & s hayes st']
stations_converter['15th & hayes st'] = stations_converter['pentagon city metro / 12th & s hayes st']
stations_converter['virginia square'] = stations_converter['virginia square metro / n monroe st & 9th st n']
stations_converter['11th & k st nw'] = stations_converter['10th & k st nw']
stations_converter['13th & u st nw'] = stations_converter['12th & u st nw']
stations_converter['17th & k st nw [formerly 17th & l st nw]'] = stations_converter['17th & k st nw']
stations_converter['18th & hayes st'] = stations_converter['aurora hills community ctr/18th & hayes st']
stations_converter['18th & bell st'] = stations_converter['crystal city metro / 18th & bell st']
stations_converter['18th & wyoming ave nw'] = stations_converter['18th st & wyoming ave nw']
stations_converter['1st & n st se'] = stations_converter['1st & n st  se']
stations_converter['20th & bell st'] = stations_converter['20th & crystal dr']
stations_converter['22nd & eads st'] = stations_converter['23rd & eads st']
stations_converter['23rd & eads'] = stations_converter['23rd & eads st']
stations_converter['26th & crystal dr'] = stations_converter['26th & s clark st']
stations_converter['34th st & minnesota ave se'] = stations_converter['randle circle & minnesota ave se']
stations_converter['4th st & massachusetts ave nw'] = stations_converter['3rd & h st nw']
stations_converter['4th st & rhode island ave ne'] = stations_converter['4th & w st ne']
stations_converter['5th st & k st nw'] = stations_converter['5th & k st nw']
stations_converter['7th & f st nw / national portrait gallery'] = stations_converter['7th & f st nw/portrait gallery']
stations_converter['8th & f st nw / national portrait gallery'] = stations_converter['7th & f st nw/portrait gallery']
stations_converter['7th & water st sw / sw waterfront'] = stations_converter['6th & water st sw / sw waterfront']
stations_converter['alta tech office'] = stations_converter['18th & m st nw']
stations_converter['bethesda ave & arlington blvd'] = stations_converter['bethesda ave & arlington rd']
stations_converter['central library'] = stations_converter['central library / n quincy st & 10th st n']
stations_converter['connecticut ave & nebraska ave nw'] = stations_converter['connecticut & nebraska ave nw']
stations_converter['court house metro / wilson blvd & n uhle st'] = stations_converter['wilson blvd & n uhle st']
stations_converter['fairfax dr & glebe rd'] = stations_converter['glebe rd & 11th st n']
stations_converter['fallsgove dr & w montgomery ave'] = stations_converter['fallsgrove dr & w montgomery ave']
stations_converter['idaho ave & newark st nw [on 2nd district patio]'] = stations_converter['wisconsin ave & newark st nw']
stations_converter['king st metro'] = stations_converter['king st metro north / cameron st']
stations_converter['14th & h st nw'] = stations_converter['14th st & new york ave nw']
stations_converter['mcpherson square - 14th & h st nw'] = stations_converter['14th st & new york ave nw']
stations_converter['mcpherson square / 14th & h st nw'] = stations_converter['14th st & new york ave nw']
stations_converter['n adams st & lee hwy'] = stations_converter['lee hwy & n adams st']
stations_converter['n fillmore st & clarendon blvd'] = stations_converter['clarendon blvd & n fillmore st']
stations_converter['n highland st & wilson blvd'] = stations_converter['clarendon metro / wilson blvd & n highland st']
stations_converter['n nelson st & lee hwy'] = stations_converter['lee hwy & n monroe st']
stations_converter['n quincy st & wilson blvd'] = stations_converter['wilson blvd & n quincy st']
stations_converter['randle circle & minnesota ave ne'] = stations_converter['randle circle & minnesota ave se']
stations_converter['s abingdon st & 36th st s'] = stations_converter['s stafford & 34th st s']
stations_converter['smithsonian / jefferson dr & 12th st sw'] = stations_converter['smithsonian-national mall / jefferson dr & 12th st sw']
stations_converter['thomas jefferson cmty ctr / 2nd st s & ivy'] = stations_converter['tj cmty ctr / 2nd st & s old glebe rd']
stations_converter['wisconsin ave & macomb st nw'] = stations_converter['wisconsin ave & newark st nw']
stations_converter['wilson blvd & n oakland st'] = stations_converter['virginia square metro / n monroe st & 9th st n']
stations_converter['mlk library/9th & g st nw'] = '31274'
stations_converter['5th & kennedy st nw'] = '31403'

def translateStations(x):
    try:
        return int(x)
    except:
        return int(stations_converter[str(x).lower().strip()])

## Translate station names to ids for dfs 5 - 18
for i in range(4, 18):
    dfs[i] = dfs[i].loc[dfs[i].start_station.notnull()]
    dfs[i] = dfs[i].loc[dfs[i].end_station.notnull()]
    dfs[i]['start_station'] = dfs[i]['start_station'].apply(translateStations)
    dfs[i]['end_station'] = dfs[i]['end_station'].apply(translateStations)

In [14]:
## Concatenate all the dfs into one large one
rides = pd.concat(dfs, axis = 0)
print rides.shape
rides.head()

(15000206, 7)


,account_type,bike_id,duration,end_datetime,end_station,start_datetime,start_station
0,Registered,W00749,0h 1min. 50sec.,4/1/2011 0:00,31103,3/31/2011 23:58,31105
1,Casual,W01048,0h 16min. 21sec.,4/1/2011 0:08,31609,3/31/2011 23:52,31224
2,Registered,W00340,0h 3min. 19sec.,3/31/2011 23:50,31611,3/31/2011 23:47,31619
3,Registered,W00981,0h 5min. 44sec.,3/31/2011 23:50,31200,3/31/2011 23:45,31202
4,Registered,W00449,0h 8min. 42sec.,3/31/2011 23:43,31201,3/31/2011 23:34,31602


In [15]:
## Fix bike ids
def bikeId(row):
    if type(row) != float:
        new_row = str(row).lower().strip()
        if len(re.findall('w[0-9]+', new_row)) == 0:
            if new_row not in blah.keys():
                blah[new_row] = 'CUSTOM_ID_'+str(len(blah))
                return blah[new_row]
            else:
                return blah[new_row]
        else:
            return new_row
    else:
        return 'NO_ID'
    
blah = {}
rides['bike_id'] = rides.bike_id.apply(bikeId)

In [16]:
## Fix account types
rides['account_type'] = rides.account_type.apply(str.lower)
rides.account_type.loc[rides.account_type == 'member'] = 1
rides.account_type.loc[rides.account_type == 'subscriber'] = 1
rides.account_type.loc[rides.account_type == 'registered'] = 1
rides.account_type.loc[rides.account_type == 'casual'] = 0

In [18]:
## Fix durations
def durationFix(row):
    try:
        new_row = str(row).lower().strip()
        check = re.findall('h', new_row)
        if len(check) > 0:
            new_row_split = new_row.split(' ')
            hours = int(new_row_split[0].split('h')[0])
            minutes = int(new_row_split[1].split('m')[0])
            seconds = int(new_row_split[2].split('s')[0])
            new_time = int(datetime.timedelta(hours = hours, minutes = minutes, seconds = seconds).total_seconds()*1000)
            return new_time
        else:
            return int(row)
    except:
        return row
    
rides['duration'] = rides.duration.apply(durationFix)

In [19]:
## Fix datetimes
def datetimeFixer(row):
    try:
        return pd.to_datetime(str(row)).strftime('%Y-%m-%d %H:%M:%S')
    except:
        return row

print "Doing start_datetime apply..."
rides['start_datetime'] = rides.start_datetime.apply(datetimeFixer)
print "Doing end_datetime apply..."
rides['end_datetime'] = rides.end_datetime.apply(datetimeFixer)

Doing start_datetime apply...
Doing end_datetime apply...


In [25]:
## Encode bike ids to unique integers
from sklearn.preprocessing import LabelEncoder
bike_id_le = LabelEncoder()
rides['bike_id'] = bike_id_le.fit_transform(rides.bike_id)
bike_ids = bike_id_le.classes_.tolist()
bike_ids_encoded = bike_id_le.transform(bike_id_le.classes_.tolist())
master_encoder_dict = {'bike_id': dict(zip(bike_ids, bike_ids_encoded)), 'account_type': {'registered': 1, 'casual': 0}}

import simplejson as json
with open('../data/historical_data/master_encoder_for_rides_historical_data.json', 'w') as f:
    json.dump(master_encoder_dict, f)

In [23]:
print rides.isnull().sum()
rides.head()

account_type       0
bike_id            0
duration           0
end_datetime       0
end_station       42
start_datetime     0
start_station      0
dtype: int64


,account_type,bike_id,duration,end_datetime,end_station,start_datetime,start_station
0,1,742,110000,2011-04-01 00:00:00,31103,2011-03-31 23:58:00,31105
1,0,1039,981000,2011-04-01 00:08:00,31609,2011-03-31 23:52:00,31224
2,1,337,199000,2011-03-31 23:50:00,31611,2011-03-31 23:47:00,31619
3,1,972,344000,2011-03-31 23:50:00,31200,2011-03-31 23:45:00,31202
4,1,443,522000,2011-03-31 23:43:00,31201,2011-03-31 23:34:00,31602


In [28]:
## Split datetimes into dates and times for use as columns on which to join holiday and weather data
print 'Doing start_time...'
rides['start_time'] = rides.start_datetime.apply(lambda x: pd.to_datetime(x).strftime('%H:%M'))
print 'Doing end_time...'
rides['end_time'] = rides.end_datetime.apply(lambda x: pd.to_datetime(x).strftime('%H:%M'))
print 'Doing start_date...'
rides['start_date'] = rides.start_datetime.apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
print 'Doing end_date...'
rides['end_date'] = rides.end_datetime.apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
rides.sort_values('start_date', axis = 0, inplace = True)
rides.head()

Doing start_time...
Doing end_time...
Doing start_date...
Doing end_date...


,account_type,bike_id,duration,end_datetime,end_station,start_datetime,start_station,start_time,end_time,start_date,end_date
149899,1,861,932000,2011-01-01 11:27:00,31216,2011-01-01 11:12:00,31103,11:12,11:27,2011-01-01,2011-01-01
149422,1,89,326000,2011-01-01 16:19:00,31105,2011-01-01 16:13:00,31101,16:13,16:19,2011-01-01,2011-01-01
149421,1,395,598000,2011-01-01 16:23:00,31110,2011-01-01 16:13:00,31106,16:13,16:23,2011-01-01,2011-01-01
149420,0,752,869000,2011-01-01 16:29:00,31216,2011-01-01 16:14:00,31219,16:14,16:29,2011-01-01,2011-01-01
149419,1,279,741000,2011-01-01 16:26:00,31500,2011-01-01 16:14:00,31401,16:14,16:26,2011-01-01,2011-01-01


In [24]:
## Save historical data to file
rides.dropna().to_csv('../data/historical_data/rides_historical_data.csv')

In [3]:
rides = pd.read_csv('../data/historical_data/rides_historical_data.csv', index_col = 0)

In [4]:
## Get historical weather
weather = pd.read_csv('../data/historical_data/weather_historical_data.csv', index_col = 0)
weather.head()

,time,precipIntensity,precipProbability,precipType,temperature,apparentTemperature,dewPoint,humidity,windSpeed,visibility,cloudCover,pressure
0,2011-01-01 00:00:00,0.0,0.0,NaN,35.34,31.1,30.28,0.82,4.88,9.49,0.03,1021.13
1,2011-01-01 00:10:00,0.0,0.0,NaN,35.34,31.1,30.28,0.82,4.88,9.49,0.03,1021.13
2,2011-01-01 00:20:00,0.0,0.0,NaN,35.34,31.1,30.28,0.82,4.88,9.49,0.03,1021.13
3,2011-01-01 00:30:00,0.0,0.0,NaN,35.34,31.1,30.28,0.82,4.88,9.49,0.03,1021.13
4,2011-01-01 00:40:00,0.0,0.0,NaN,35.34,31.1,30.28,0.82,4.88,9.49,0.03,1021.13


In [5]:
## Get holidays
holidays = pd.read_csv('../data/holidays.csv', index_col = 0)
holidays.head()

,date,holiday
index,,
0,2017-01-02,New Year's Day
1,2017-01-16,Martin Luther King Jr. Day
2,2017-01-20,Inauguration Day
3,2017-02-20,Washington's Birthday
4,2017-04-17,DC Emancipation Day


In [6]:
## We need to join rides and weather on datetime, but weather is in intervals of 10 min while rides is in intervals of 1 min.
## To fix this we will make temporary columns of the datetime without the ones minute place in each df and join on that.
weather['datetime'] = weather.time.apply(lambda x: x[0:-4])
weather.drop('time', axis = 1, inplace = True)
rides['datetime'] = rides['start_datetime'].apply(lambda x: x[0:-4])
## Now join on datetime
rides_weath = rides.merge(weather, on = 'datetime', how = 'left')
rides_weath.head()

,account_type,bike_id,duration,end_datetime,end_station,start_datetime,start_station,start_time,end_time,start_date,...,precipProbability,precipType,temperature,apparentTemperature,dewPoint,humidity,windSpeed,visibility,cloudCover,pressure
0,1,861,932000,2011-01-01 11:27:00,31216.0,2011-01-01 11:12:00,31103,11:12,11:27,2011-01-01,...,0.00,NaN,48.73,44.88,41.84,0.77,8.85,9.59,0.47,1019.99
1,1,89,326000,2011-01-01 16:19:00,31105.0,2011-01-01 16:13:00,31101,16:13,16:19,2011-01-01,...,0.01,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41
2,1,395,598000,2011-01-01 16:23:00,31110.0,2011-01-01 16:13:00,31106,16:13,16:23,2011-01-01,...,0.01,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41
3,0,752,869000,2011-01-01 16:29:00,31216.0,2011-01-01 16:14:00,31219,16:14,16:29,2011-01-01,...,0.01,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41
4,1,279,741000,2011-01-01 16:26:00,31500.0,2011-01-01 16:14:00,31401,16:14,16:26,2011-01-01,...,0.01,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41


In [7]:
## Now join the rides and weather combined df with holidays on datetime after renaming date to datetime in holidays
holidays.columns = ['start_date', 'holiday']
rides_weath_hol = rides_weath.merge(holidays, on = 'start_date', how = 'left')
rides_weath_hol['holiday'] = rides_weath_hol.holiday.apply(lambda x: 0 if pd.isnull(x) else 1)
df = rides_weath_hol
df['precipType'].loc[df.precipType.isnull()] = 'none'
df.head()

/usr/lib64/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,account_type,bike_id,duration,end_datetime,end_station,start_datetime,start_station,start_time,end_time,start_date,...,precipType,temperature,apparentTemperature,dewPoint,humidity,windSpeed,visibility,cloudCover,pressure,holiday
0,1,861,932000,2011-01-01 11:27:00,31216.0,2011-01-01 11:12:00,31103,11:12,11:27,2011-01-01,...,none,48.73,44.88,41.84,0.77,8.85,9.59,0.47,1019.99,0
1,1,89,326000,2011-01-01 16:19:00,31105.0,2011-01-01 16:13:00,31101,16:13,16:19,2011-01-01,...,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41,0
2,1,395,598000,2011-01-01 16:23:00,31110.0,2011-01-01 16:13:00,31106,16:13,16:23,2011-01-01,...,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41,0
3,0,752,869000,2011-01-01 16:29:00,31216.0,2011-01-01 16:14:00,31219,16:14,16:29,2011-01-01,...,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41,0
4,1,279,741000,2011-01-01 16:26:00,31500.0,2011-01-01 16:14:00,31401,16:14,16:26,2011-01-01,...,rain,53.39,53.39,47.58,0.81,8.38,9.89,0.84,1016.41,0


In [8]:
## Add working_day column where any weekday that is not a holiday is a working day and all others are not
print "Doing working_day..."
df['working_day'] = df['start_date'].apply(lambda x: 1 if pd.to_datetime(x).weekday() < 5 else 0)
df['working_day'].loc[df['holiday'] == 1] = 0
## Make a year column
print "Doing year..."
df['year'] = df.start_date.apply(lambda x: int(x[0:4]))
## Make a month column
print "Doing month..."
df['month'] = df.start_date.apply(lambda x: int(x[5:7]))
## Make a season column where months 1-3 are winter, 4-6 are spring, 7-9 are summer, and 10-12 are fall
print "Doing season..."
df['season'] = df.month.apply(lambda x: 'Winter' if x < 4 else 'Spring' if x > 3 and x < 7 else 'Summer' if x > 6 and x < 10 else 'Fall')
## Make an hour of the day column
print "Doing hour..."
df['hour'] = df.start_time.apply(lambda x: x[0:2])
## Round the temperature for later groupby statements
print "Doing apparentTemperatureRounded..."
df['apparentTemperatureRounded'] = df.apparentTemperature.apply(lambda x: np.round(x))
## Round the windSpeed for later groupby statements
print "Doing windSpeedRounded..."
df['windSpeedRounded'] = df.windSpeed.apply(lambda x: np.round(x))

Doing working_day...
Doing year...
Doing month...
Doing season...
Doing hour...
Doing apparentTemperatureRounded...
Doing windSpeedRounded...


In [9]:
## Save the data for use later
df.to_csv('../data/historical_data/master_historical_data.csv')